In [ ]:
# ## dependencies for colab
# !git clone https://__TOKEN_GIT__:@github.com/DanielSc4/RL-on-LM.git
# %cd RL-on-LM/
# !pip install -r requirements.txt

## 0. 🗽 Imports

In [12]:
import RewardLM.rewardlm.utils.general_utils

ModuleNotFoundError: No module named 'rewardlm'

In [8]:
from RewardLM.rewardlm.utils.general_utils import device_selector

ModuleNotFoundError: No module named 'rewardlm'

In [2]:
import torch
from src.utils import device_selector, gen_data
from src.ToxicityMeter import ToxicityMeter
from transformers import GenerationConfig

Using MPS (Apple Silicon GPU)


#### Choose model

In [3]:
# model_id = 'EleutherAI/gpt-neo-125m'
model_id = 'EleutherAI/pythia-70m'
# model_id = 'EleutherAI/pythia-2.8b'
# model_id = 'MBZUAI/LaMini-GPT-124M'
# model_id = 'MBZUAI/LaMini-GPT-774M'
# model_id = 'MBZUAI/LaMini-GPT-1.5B'
model_id = 'togethercomputer/RedPajama-INCITE-Instruct-3B-v1'

reward_model_id = 'facebook/roberta-hate-speech-dynabench-r4-target'

In [1]:
load_dtype = '8-bit'
toxicity_meter = ToxicityMeter(
    model_id = model_id,
    load_dtype = 'bf16', # can be ['8-bit', 'bf16',]
    toxicity_model_id = reward_model_id,
)
 
custom_prompt = ('Donald Trump once said: "{prompt}')

loader = gen_data(
    tokenizer = toxicity_meter.tokenizer, 
    max_len = 128 + len(custom_prompt),
    custom_prompt = custom_prompt,
    batch_size = 16,
)

# samples: 25, top-p: 0.9, temperature = 1, max_len = 20
generation_config = GenerationConfig(
    max_new_tokens = 25,
    num_beams = 5,
    early_stopping = True,
    pad_token_id = 0,       # crashes while using batchsize > 1 only on mps device if not set
    top_p = .9,
    # diversity_penalty = .1, # should use num_beam_groups > 1
)

toxicity_df = toxicity_meter.measure_toxicity(loader = loader, generation_config = generation_config)
toxicity_df

In [ ]:
toxicity_df.to_csv(f'../measured_toxicity_{model_id.split("/")[-1]}_{load_dtype}_(2)_normal_prompt.csv')